## **Calculating the surface-forced overturning stream function in potential density coordinates**

### **Description**

Recipe showing how to calculate the surface-forced overturning stream function in potential density-coordinates using annual-mean outputs from the National Oceanography Centre Near-Present-Day global eORCA1 configuration of NEMO forced using JRA55-do from 1976-2024.

For more details on this model configuration and the available outputs, users can explore the Near-Present-Day documentation [here](https://noc-msm.github.io/NOC_Near_Present_Day/).

In [1]:
# -- Import required Python packages -- #
import gsw
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# -- Import nemo_cookbook tools -- #
from nemo_cookbook import compute_sfoc_sigma0

### **Using Dask**

**Optional: Connect Client to Dask Local Cluster to run analysis in parallel.**

Note that, although using Dask is not strictly necessary for this simple example using eORCA1, if we wanted to generalise this recipe to eORCA025 or eORCA12 outputs, using Dask would be essential to avoid unnecessary slow calculations using only a single process.

In [ ]:
# -- Initialise Dask Local Cluster -- #
import dask
from dask.distributed import Client, LocalCluster

# Update temporarty directory for Dask workers:
dask.config.set({'temporary_directory': '/home/otooth/work/Diagnostics/proj_NPD_diag/nemo_cookbook/recipes',
                 'local_directory': '/home/otooth/work/Diagnostics/proj_NPD_diag/nemo_cookbook/recipes'
                 })

# Create Local Cluster:
cluster = LocalCluster(n_workers=4, threads_per_worker=5, memory_limit='5GB')
client = Client(cluster)
client

### **Preparing NEMO Model Data**
**Let's begin by loading the grid variables for our eORCA1 NEMO model from the [JASMIN Object Store](https://help.jasmin.ac.uk/docs/short-term-project-storage/using-the-jasmin-object-store/)**. 

**Alternatively, you can replace the ``domain_filepath`` below with a file path to your domain_cfg.nc file and read this with xarray's ``open_dataset()`` function.**

In [ ]:
# -- Import eORCA1 domain data -- #
# Define directory path to ancillary files:
domain_filepath = "https://noc-msm-o.s3-ext.jc.rl.ac.uk/npd-eorca1-jra55v1/domain"

# Open eORCA1 model grid data:
ds_domain = xr.open_zarr(domain_filepath, consolidated=True, chunks={})

# Extract zonal grid cell widths (m):
e1t = ds_domain['e1t'].squeeze()
# Extract meridional grid cell widths (m):
e2t = ds_domain['e2t'].squeeze()
# Extract Atlantic Ocean mask:
atl_mask = ds_domain['atlmsk'].squeeze()

**Next, we need to import surface heat and freshwater fluxes stored at T-points**

In [ ]:
# -- Import eORCA1 tracer data -- #
# Define directory path to model output files:
output_dir = "https://noc-msm-o.s3-ext.jc.rl.ac.uk/npd-eorca1-jra55v1"

# Extract net downward surface heat flux (W/m2):
thetao_con = xr.open_zarr(f"{output_dir}/T1y/thetao_con", consolidated=True, chunks={})['thetao_con']
# Extract absolute salinity (g/kg):
so_abs = xr.open_zarr(f"{output_dir}/T1y/so_abs", consolidated=True, chunks={})['so_abs']

**Next, we need to import the sea surface temperature and sea surface salinity stored at T-points**

In [ ]:
# -- Import eORCA1 tracer data -- #
# Define directory path to model output files:
output_dir = "https://noc-msm-o.s3-ext.jc.rl.ac.uk/npd-eorca1-jra55v1"

# Extract conservative temperature (C):
tos_con = xr.open_zarr(f"{output_dir}/T1y/tos_con", consolidated=True, chunks={})['tos_con']
# Extract absolute salinity (g/kg):
sos_abs = xr.open_zarr(f"{output_dir}/T1y/sos_abs", consolidated=True, chunks={})['sos_abs']

# Calculate potential density anomaly referenced to the sea surface (kg/m3):
sigma0 = gsw.density.sigma0(CT=tos_con, SA=sos_abs)
sigma0.name = 'sigma0'

In [ ]:
# -- Import eORCA1 velocity data -- #
# Extract vertical grid cell thicknesses (m):
e3v = xr.open_zarr(f"{output_dir}/V1y/e3v", consolidated=True, chunks={})['e3v']
# Extract meridional velocities (m/s):
vo = xr.open_zarr(f"{output_dir}/V1y/vo", consolidated=True, chunks={})['vo']

### **Calculating Surface-Forced Overturning Stream Function**

**Now all our input variables are ready, let's calculate the Surface-Forced Overturning Stream Function in density-coordinates**

In [ ]:
# -- Create Task: Compute Atlantic Meridional Overturning Circulation (MOC_sigma0) in density-coordinates -- #
# Apply the Atlantic Ocean sector mask and accumulate from the lightest to the densest isopycnal surface:
moc_sigma0_atl = compute_moc_tracer(vo=vo,
                                    e1v=e1v,
                                    e3v=e3v,
                                    tracer=sigma0,
                                    tracer_bins=np.arange(21, 29, 0.01),
                                    dir = '+1',
                                    mask=atl_mask,
                                    )

# Notice that the output is a dask array, so we haven't actually computed the MOC_sigma0 yet.
moc_sigma0_atl

In [ ]:
# -- Complete Task: Compute Meridional Overturning Circulation (MOC_sigma0) in density-coordinates -- #
moc_sigma0_atl = moc_sigma0_atl.compute()

In [ ]:
# -- Plot time-mean MOC_sigma0 -- #
moc_sigma0_atl.mean(dim='time_counter').plot(yincrease=False)